In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Hugging Face DLCs: Fine-tuning Gemma with Transformer Reinforcement Learning (TRL) on Vertex AI

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/open-models/fine-tuning/vertex_ai_trl_fine_tuning_gemma.ipynb">
      <img width="32px" src="https://www.gstatic.com/pantheon/images/bigquery/welcome_page/colab-logo.svg" alt="Google Colaboratory logo"><br> Open in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fgenerative-ai%2Fmain%2Fopen-models%2Ffine-tuning%2Fvertex_ai_trl_fine_tuning_gemma.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"><br> Open in Colab Enterprise
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/main/open-models/fine-tuning/vertex_ai_trl_fine_tuning_gemma.ipynb">
      <img src="https://www.gstatic.com/images/branding/gcpiconscolors/vertexai/v1/32px.svg" alt="Vertex AI logo"><br> Open in Vertex AI Workbench
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/open-models/fine-tuning/vertex_ai_trl_fine_tuning_gemma.ipynb">
      <img width="32px" src="https://upload.wikimedia.org/wikipedia/commons/9/91/Octicons-mark-github.svg" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
</table>

![Hugging Face x Google Cloud](https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/google-cloud/thumbnail.png)

| | |
|-|-|
| Author(s) | [Alvaro Bartolome](https://github.com/alvarobartt), [Philipp Schmid](https://github.com/philschmid), [Simon Pagezy](https://github.com/pagezyhf), and [Jeff Boudier](https://github.com/jeffboudier) |

## Overview

> [**Gemma**](https://ai.google.dev/gemma) is a family of lightweight, state-of-the-art open models built from the same research and technology used to create the Gemini models, developed by Google DeepMind and other teams across Google.

> [**Transformer Reinforcement Learning (TRL)**](https://github.com/huggingface/trl) is a framework developed by Hugging Face to fine-tune and align both transformer language and diffusion models using methods such as Supervised Fine-Tuning (SFT), Reward Modeling (RM), Proximal Policy Optimization (PPO), Direct Preference Optimization (DPO), and others.

> [**Hugging Face DLCs**](https://github.com/huggingface/Google-Cloud-Containers) are pre-built and optimized Deep Learning Containers (DLCs) maintained by Hugging Face and Google Cloud teams to simplify environment configuration for your ML workloads.

> [**Google Vertex AI**](https://cloud.google.com/vertex-ai) is a Machine Learning (ML) platform that lets you train and deploy ML models and AI applications, and customize large language models (LLMs) for use in your AI-powered applications.

This notebook showcases how to fine-tune Google Gemma with Supervised Fine-Tuning (SFT) and Low-Rank Adaptation (LoRA) in a single GPU via a custom job on Vertex AI using the Hugging Face PyTorch Deep Learning Container (DLC) for Training.

By the end of this notebook, you will learn:

- About Hugging Face's TRL and LLM fine-tuning
- How to create and run a custom container job on Vertex AI

## Get started

### Install Vertex AI SDK and other required packages

To run this example, you will only need the [`google-cloud-aiplatform`](https://github.com/googleapis/python-aiplatform) Python SDK and the [`huggingface_hub`](https://github.com/huggingface/huggingface_hub) Python package.

In [ ]:
%pip install --upgrade --user --quiet google-cloud-aiplatform huggingface_hub

### Restart runtime (Colab only)

To use the newly installed packages in this Jupyter environment, if you are on Colab you must restart the runtime. You can do this by running the cell below, which restarts the current kernel. The restart might take a minute or longer. After it's restarted, continue to the next step.

In [ ]:
# Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

<div class="alert alert-block alert-warning">
<b>⚠️ The kernel is going to restart. Wait until it's finished before continuing to the next step. ⚠️</b>
</div>


### Authenticate your Google Cloud account

Depending on your Jupyter environment, you may have to manually authenticate. Follow the relevant instructions below.

**1. Vertex AI Workbench**

* Do nothing as you are already authenticated.

**2. Local JupyterLab instance, uncomment and run:**

In [ ]:
# !gcloud auth login

**3. Colab, uncomment and run:**

In [ ]:
# from google.colab import auth
# auth.authenticate_user()

### Authenticate your Hugging Face account

As [`google/gemma-2b`](https://huggingface.co/google/gemma-2b) is a gated model, you need to have a Hugging Face Hub account, and accept the Google's usage license for Gemma. Once that's done, you need to generate a new user access token with read-only access so that the weights can be downloaded from the Hub in the Hugging Face DLC for TGI.

> Note that the user access token can only be generated via [the Hugging Face Hub UI](https://huggingface.co/settings/tokens/new), where you can either select read-only access to your account, or follow the recommendations and generate a fine-grained token with read-only access to [`google/gemma-2b`](https://huggingface.co/google/gemma-2b).

Then you can install the `huggingface_hub` that comes with a CLI that will be used for the authentication with the token generated in advance. So that then the token can be safely retrieved via `huggingface_hub.get_token`.

In [ ]:
from huggingface_hub import interpreter_login

interpreter_login()

Read more about [Hugging Face Security](https://huggingface.co/docs/hub/en/security), specifically about [Hugging Face User Access Tokens](https://huggingface.co/docs/hub/en/security-tokens).

### Set Google Cloud environment variables

You will need to set the following environment variables that are required to run this example.

In [ ]:
import os

PROJECT_ID = "[your-project-id]"  # @param {type:"string", isTemplate: true}
if PROJECT_ID == "[your-project-id]":
    PROJECT_ID = str(os.environ.get("GOOGLE_CLOUD_PROJECT"))
os.environ["PROJECT_ID"] = PROJECT_ID

os.environ["LOCATION"] = os.environ.get("GOOGLE_CLOUD_REGION", "us-central1")

### (Optional) Create a bucket in Google Cloud Storage (GCS)

As the custom job on Vertex AI will need to dump the fine-tuned artifacts somewhere on Google Cloud, you will need to have a bucket available on Google Cloud Storage (GCS), so that you can specify it as the bucket to be used within the custom job, so that anything written within the container in that path is automatically uploaded to GCS.

In [ ]:
BUCKET_URI = "[your-bucket-uri]"  # @param {type:"string", isTemplate: true}
if BUCKET_URI == "[your-bucket-uri]":
    raise ValueError(
        "A valid BUCKET_URI (e.g. `gs://path/to/bucket`) needs to be specified"
    )
os.environ["BUCKET_URI"] = BUCKET_URI

> Uncomment the `gcloud storage buckets create` command below if you need to create a bucket on GCS.

In [ ]:
# !gcloud storage buckets create $BUCKET_URI --project $PROJECT_ID --location=$LOCATION --default-storage-class=STANDARD --uniform-bucket-level-access

### Initialize Vertex AI SDK

To get started using Vertex AI, you must have an existing Google Cloud project and [enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com), if not enabled already.

Learn more about [setting up a project and a development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment).

In [ ]:
aiplatform.init(
    project=os.getenv("PROJECT_ID"),
    location=os.getenv("LOCATION"),
    staging_bucket=os.getenv("BUCKET_URI"),
)

## Requirements

You will need to have the following IAM roles set:

- Artifact Registry Reader (roles/artifactregistry.reader)
- Vertex AI User (roles/aiplatform.user)
- Storage Object Creator (roles/storage.objectCreator)

For more information about granting roles, see [Manage access](https://cloud.google.com/iam/docs/granting-changing-revoking-access).

---

You will also need to enable the following APIs (if not enabled already):

- Vertex AI API (aiplatform.googleapis.com)
- Cloud Storage API (storage-api.googleapis.com)
- Artifact Registry API (artifactregistry.googleapis.com)

For more information about API enablement, see [Enabling APIs](https://cloud.google.com/apis/docs/getting-started#enabling_apis).

---

You will also need to have a Google Cloud Storage (GCS) bucket where the custom job can read and write artifacts.

---

To access Gemma on Hugging Face, you're required to review and agree to Google's usage license on the Hugging Face Hub for any of the models from the [Gemma release collection](https://huggingface.co/collections/google/gemma-release-65d5efbccdbb8c4202ec078b), and the access request will be processed immediately.

## Create a `CustomContainerTrainingJob` on Vertex AI

You need to define a `CustomContainerTrainingJob` that runs on top of the Hugging Face PyTorch DLC for Training.

The Hugging Face PyTorch DCL for Training comes with most of the Hugging Face Python libraries installed, so as to provide a seamless environment to use the Hugging Face stack on Google Cloud. In this example, as already mentioned, [`trl`](https://github.com/huggingface/trl) will be used to run the Supervised Fine-Tuning (SFT) on top of [`transformers`](https://github.com/huggingface/transformers) for the modelling, [`peft`](https://github.com/huggingface/peft) for the LoRA support and [`bitsandbytes`](https://github.com/huggingface/bitsandbytes) for the quantization support.

Since the Hugging Face PyTorch DLC for Training does not have any pre-defined `CMD` or `ENTRYPOINT`, you will need to set it to `trl sft`; which is the TRL command to run the SFT fine-tuning via the recently released [TRL CLI](https://huggingface.co/docs/trl/en/clis).

In [ ]:
job = aiplatform.CustomContainerTrainingJob(
    display_name="gemma-2b-sft-lora",
    container_uri="us-docker.pkg.dev/deeplearning-platform-release/gcr.io/huggingface-pytorch-training-cu121.2-3.transformers.4-42.ubuntu2204.py310",
    command="trl sft",
)

## Prepare `CustomContainerTrainingJob` on Vertex AI

Before submitting the `CustomContainerTrainingJob` you need to define the following:

### Model to fine-tune

In this case you will be fine-tuning a base model, [`google/gemma-2b`](https://huggingface.co/google/gemma-2b), meaning that the model by itself will just generate text and has not been fined-tuned in advance i.e. no prompt templates, etc.

### Model requirements

Fine-tuning a model can be expensive, you can estimate that to full fine-tune an LLM in half precision you would need around four times the disk size of the LLM in GPU VRAM.

For a 2B LLM that takes around 5GiB on disk, it could translate into 20GiB for the half precision fine-tuning; but that could be reduced further by using a quantized optimizer, using LoRA or QLoRA, reducing the batch size, etc. In this case, you will be using an L4 NVIDIA GPU that comes with 24GiB of VRAM so both SFT or SFT + LoRA will work out of the box as those fit within the 24GiB VRAM limit.

Read more about it in [Eleuther AI - Transformer Math 101](https://blog.eleuther.ai/transformer-math/).

### Dataset to fine-tune on

SFT expects either a text-only, a conversational, or a prompt-completion dataset; in this case, a text-only dataset [`timdettmers/openassistant-guanaco`](https://huggingface.co/datasets/timdettmers/openassistant-guanaco) will be used.

This dataset is a subset of the [Open Assistant dataset](https://huggingface.co/datasets/OpenAssistant/oasst1); and this subset only contains the highest-rated paths in the conversation tree, with a total of 9,846 samples.

## Submit `CustomContainerTrainingJob` on Vertex AI

As the `CustomContainerTrainingJob` defines the command `trl sft` the arguments to be provided are listed either in the Python reference at [trl.SFTConfig](https://huggingface.co/docs/trl/en/sft_trainer#trl.SFTConfig) or via the `trl sft --help` command.

Read more about the [TRL CLI](https://huggingface.co/docs/trl/en/clis).

In [ ]:
args = [
    # MODEL
    "--model_name_or_path=google/gemma-2b",
    "--torch_dtype=bfloat16",
    "--attn_implementation=flash_attention_2",
    # DATASET
    "--dataset_name=timdettmers/openassistant-guanaco",
    "--dataset_text_field=text",
    # PEFT
    "--use_peft",
    "--lora_r=16",
    "--lora_alpha=32",
    "--lora_dropout=0.1",
    "--lora_target_modules=all-linear",
    # TRAINER
    "--bf16",
    "--max_seq_length=1024",
    "--per_device_train_batch_size=8",
    "--gradient_accumulation_steps=4",
    "--gradient_checkpointing",
    "--learning_rate=0.0002",
    "--lr_scheduler_type=cosine",
    "--optim=adamw_bnb_8bit",
    "--num_train_epochs=1",
    "--logging_steps=10",
    "--do_eval",
    "--eval_steps=100",
    "--report_to=none",
    f"--output_dir={os.getenv('BUCKET_URI').replace('gs://', '/gcs/')}/gemma-2b-sft-lora",
    "--overwrite_output_dir",
    "--seed=42",
    "--log_level=debug",
]

> It's important to note that since GCS FUSE is used to mount the bucket as a directory within the running container job, the mounted path follows the formatting `/gcs/<BUCKET_NAME>`; meaning that anything the `SFTTrainer` writes there will be automatically uploaded to the GCS Bucket.
>
> More information in the [Vertex AI Documentation - Prepare training code](https://cloud.google.com/vertex-ai/docs/training/code-requirements).

Then you need to call the `submit` method on the `aiplatform.CustomContainerTrainingJob`, which is a non-blocking method that will schedule the job without blocking the execution.

The arguments provided to the `submit` method are listed below:

* **`args`** defines the list of arguments to be provided to the `trl sft` command, provided as `trl sft --arg_1=value ...`.

* **`replica_count`** defines the number of replicas to run the job in, for training normally this value will be set to one.

* **`machine_type`**, **`accelerator_type`** and **`accelerator_count`** define the machine i.e. Compute Engine instance, the accelerator (if any), and the number of accelerators (ranging from 1 to 8); respectively. The `machine_type` and the `accelerator_type` are tied together, so you will need to select an instance that supports the accelerator that you are using and vice-versa. More information about the different instances at [Compute Engine Documentation - GPU machine types](https://cloud.google.com/compute/docs/gpus), and about the `accelerator_type` naming at [Vertex AI Documentation - MachineSpec](https://cloud.google.com/vertex-ai/docs/reference/rest/v1/MachineSpec).

* **`base_output_dir`** defines the base directory that will be mounted within the running container from the GCS Bucket, conditioned by the `staging_bucket` argument provided to the `aiplatform.init` initially.

* (optional) **`environment_variables`** defines the environment variables to define within the running container. As you are fine-tuning a gated model i.e. [`google/gemma-2b`](https://huggingface.co/google/gemma-2b), you need to set the `HF_TOKEN` environment variable. Additionally, some other environment variables are defined to set the cache path (`HF_HOME`) and to ensure that the logging messages are streamed to Google Cloud Logs Explorer properly (`TRL_USE_RICH`, `ACCELERATE_LOG_LEVEL`, `TRANSFORMERS_LOG_LEVEL`, and `TQDM_POSITION`).

* (optional) **`timeout`** and **`create_request_timeout`** define the timeouts in seconds before interrupting the job execution or the job creation request (time to allocate required resources and start the execution), respectively.

In [ ]:
from huggingface_hub import get_token

job.submit(
    args=args,
    replica_count=1,
    machine_type="g2-standard-12",
    accelerator_type="NVIDIA_L4",
    accelerator_count=1,
    base_output_dir=f"{os.getenv('BUCKET_URI')}/gemma-2b-sft-lora",
    environment_variables={
        "HF_HOME": "/root/.cache/huggingface",
        "HF_TOKEN": get_token(),
        "TRL_USE_RICH": "0",
        "ACCELERATE_LOG_LEVEL": "INFO",
        "TRANSFORMERS_LOG_LEVEL": "INFO",
        "TQDM_POSITION": "-1",
    },
    timeout=60 * 60 * 3,  # 3 hours (10800s)
    create_request_timeout=60 * 10,  # 10 minutes (600s)
)

> The `CustomContainerTrainingJob` will run asynchronously, meaning that the execution won't be blocked and the job will automatically allocate and deallocate the required resources. So once triggered, you can go to Vertex AI in the Google Cloud Console to monitor the job closely.

## What's next?

Once the fine-tuning is done, you can already deploy it!

To deploy it, you can either [merge the LoRA adapters into the fine-tuned Google Gemma model](https://huggingface.co/docs/peft/en/developer_guides/model_merging) in advance, or just [deploy the base model and the adapter separately](https://huggingface.co/blog/multi-lora-serving).

As deployment and serving is out of the scope of this example, you can refer to the following examples in [`GoogleCloudPlatform/generative-ai`](https://github.com/GoogleCloudPlatform/generative-ai):

- [Hugging Face DLCs: Serving Gemma with Text Generation Inference (TGI) on Vertex AI](./vertex_ai_text_generation_inference_gemma.ipynb)

Or to the examples available within the [`huggingface/Google-Cloud-Containers`](https://github.com/huggingface/Google-Cloud-Containers) repository:

- [Deploy Gemma 7B with TGI on Vertex AI](https://github.com/huggingface/Google-Cloud-Containers/blob/main/examples/vertex-ai/notebooks/deploy-gemma-on-vertex-ai/vertex-notebook.ipynb)
- [Deploy Gemma 7B from GCS with TGI on Vertex AI](https://github.com/huggingface/Google-Cloud-Containers/blob/main/examples/vertex-ai/notebooks/deploy-gemma-from-gcs-on-vertex-ai/vertex-notebook.ipynb)

## References

- [GitHub Repository - Hugging Face DLCs for Google Cloud](https://github.com/huggingface/Google-Cloud-Containers): contains all the containers developed by the collaboration of both Hugging Face and Google Cloud teams; as well as a lot of examples on both training and inference, covering both CPU and GPU, as well support for most of the models within the Hugging Face Hub.
- [Google Cloud Documentation - Hugging Face DLCs](https://cloud.google.com/deep-learning-containers/docs/choosing-container#hugging-face): contains a table with the latest released Hugging Face DLCs on Google Cloud.
- [Google Artifact Registry - Hugging Face DLCs](https://console.cloud.google.com/artifacts/docker/deeplearning-platform-release/us/gcr.io): contains all the DLCs released by Google Cloud that can be used.
- [Hugging Face Documentation - Google Cloud](https://huggingface.co/docs/google-cloud): contains the official Hugging Face documentation for the Google Cloud DLCs.